# Imports

In [2]:
from dash import Dash, dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template
import plotly.express as px
import pandas 
import numpy as np

# df_edu

In [3]:
df_edu = pandas.read_csv('data/states_all.csv').iloc[:, 1:]
df_edu.head()

,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,735036.0,...,57948.0,58025.0,41167.0,NaN,NaN,731634.0,208.0,252.0,207.0,NaN
1,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,350902.0,...,9748.0,8789.0,6714.0,NaN,NaN,122487.0,NaN,NaN,NaN,NaN
2,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,1007732.0,...,55433.0,49081.0,37410.0,NaN,NaN,673477.0,215.0,265.0,209.0,NaN
3,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,483488.0,...,34632.0,36011.0,27651.0,NaN,NaN,441490.0,210.0,256.0,211.0,NaN
4,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,8520926.0,...,418418.0,363296.0,270675.0,NaN,NaN,5254844.0,208.0,261.0,202.0,NaN


# df_ski

In [12]:
df_ski = pandas.read_csv('data/European_Ski_Resorts.csv') #.drop("Unnamed: 0", axis=1)

# Data Table 1

In [13]:
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
app = Dash(__name__, ) # external_stylesheets=[dbc.themes.BOOTSTRAP]

app.layout = html.Div(
    dash_table.DataTable(
        columns=[{"name": i, "id": i} for i in df_edu.columns],
        data=df_edu.to_dict('records'),
        filter_action="native",
        sort_action="native",
        export_format="csv",
        style_header={
            'backgroundColor': 'rgb(30, 30, 30)',
            'fontWeight': 'bold',
            'font-family': 'Arial',
            'color': 'lightgrey'
        },
        style_data={
            'font-family': 'Arial',
            'color': 'grey',
            'backgroundColor': 'rgb(50, 50, 50)',
        }
    )
)

app.run_server(debug=True)
               

# ASSIGNMENT: Data Table

In [19]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H2(id="title", style={"text-align": "center"}),
    html.P("Select Country Below:"),
    dcc.Dropdown(
        id="country_dropdown",
        options=df_ski['Country'].unique(),
        value="Andorra",
        placeholder="Select a country",
        className="dbc"
    ),
    html.Br(), html.Br(),
    html.P("Select Elevation:"),
    dcc.Slider(
        id="elevation_slider",
        min=0,
        max=4000,
        step=500,
        value=500,
        marks={i: {"label": f"{i} m", "style": {"fontSize": 16}}
               for i in range(0, 4001, 500)},
        className="dbc"
    ),
    html.Br(), html.Br(),
    html.Div(id="output_div")
    
])

@app.callback(
    Output("title", "children"),
    Output("output_div", "children"),
    Input("country_dropdown", "value"),
    Input("elevation_slider", "value"),
)
def elevation_table(country: str, elevation: int):
    if not country and not elevation:
        raise PreventUpdate

    df = df_ski.query("HighestPoint > @elevation and Country == @country")
    
    title = f"Ski resorts in {country} with elevation > {elevation}"
    
    table = dash_table.DataTable(
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        filter_action="native",
        sort_action="native",
        export_format="csv",
        style_header={
            'backgroundColor': 'rgb(30, 30, 30)',
            'fontWeight': 'bold',
            'font-family': 'Arial',
            'color': 'lightgrey'
        },
        style_data={
            'font-family': 'Arial',
            'color': 'grey',
            'backgroundColor': 'rgb(50, 50, 50)',
        }
    )
    
    return title, table 

app.run_server(debug=True)